# Load libraries

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import autokeras as ak
from autokeras.image_supervised import load_image_dataset

In [ ]:
# modified from
# https://thispointer.com/python-how-to-get-list-of-files-in-directory-and-sub-directories/
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = [item for item in os.listdir(dirName) if not item.startswith('.')]
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
            
    allFiles = [each for each in allFiles if each.endswith((".jpg", ".jpeg", ".png"))]           
    return allFiles

# Auto-Keras

Auto-Keras is an open source software library for automated machine learning. Auto-Keras provides functions to automatically search for architecture and hyperparameters of deep learning models.

https://github.com/jhfjhfj1/autokeras

https://autokeras.com/start/

## Prepare csv file

In [ ]:
train_imgs = getListOfFiles("/keras2production/fruits/Training")    
train_imgs_df = pd.DataFrame(train_imgs, columns=["File"])
train_imgs_df["Label"] = train_imgs_df.apply(lambda row: re.sub('/keras2production/fruits/Training/', '', 
                                                     os.path.dirname(row.File)), axis=1)
train_imgs_df.head()

In [ ]:
test_imgs = getListOfFiles("/keras2production/fruits/Test")    
test_imgs_df = pd.DataFrame(test_imgs, columns=["File"])
test_imgs_df["Label"] = test_imgs_df.apply(lambda row: re.sub('/keras2production/fruits/Test/', '', 
                                                     os.path.dirname(row.File)), axis=1)
test_imgs_df.head()

In [ ]:
train_imgs_df.to_csv('/keras2production/fruits/train_labels.csv', index=False)
test_imgs_df.to_csv('/keras2production/fruits/test_labels.csv', index=False)

## Prepare classifier for raw images

In [ ]:
clf = ak.ImageClassifier(verbose=True, augment=False)

In [ ]:
x_train, y_train = load_image_dataset(csv_file_path="/keras2production/fruits/train_labels.csv",
                                      images_path="/keras2production/fruits/Training/")
print(x_train.shape)
print(y_train.shape)

In [ ]:
x_test, y_test = load_image_dataset(csv_file_path="/keras2production/fruits/test_labels.csv",
                                    images_path="/keras2production/fruits/Test/")

print(x_test.shape)
print(y_test.shape)

## Run fit

In [ ]:
clf.fit(x_train, y_train, time_limit=60)

In [ ]:
# running a final training after found the best architecture
clf.final_fit(x_train, y_train, x_test, y_test, retrain=True)

results = clf.predict(x_test)

y = clf.evaluate(x_test, y_test)
print(y * 100)